In [2]:
# Importing libraries 
import numpy as np
import matplotlib.pyplot as plt
from cvxpy import *
import pandas as pd 

%matplotlib inline

In [6]:
# In the next few cells we need to import price data and then 
# using if-else statements create Time of Use pricing on the timeseries
# Using EvGO pay as you go data to create time of use prices in 24 hr format


# Create an empty list to store the time-of-use prices
tou_prices = []

# Loop through all the hours in a day (0-23)
for hour in range(24):
    if 0 <= hour < 8:  # 12-8am
        price = 0.39
    elif 8 <= hour < 16:  # 8am-4pm
        price = 0.51
    elif 16 <= hour < 21:  # 4pm-9pm
        price = 0.59
    else:  # 9pm-12am
        price = 0.51
    
    # Append the hour and price to the list
    tou_prices.append([hour, price])

# Create a data frame using the tou_prices list
df_tou_prices = pd.DataFrame(tou_prices, columns=['Hour', 'Price'])

# Display the data frame
print(df)


    Departure_Time  Arrival_Time  Energy_Consumed
0                6             9        31.560570
1               19            22        23.040538
2               14            15        54.050748
3               10            13        98.708541
4                7            10        31.784974
..             ...           ...              ...
95               6             8        48.536573
96               8            12        96.998934
97              23             4        96.725798
98               0             1        86.770851
99              11            12        36.500400

[100 rows x 3 columns]


In [24]:
 ## import pandas as pd
import numpy as np

# Set the seed for reproducibility
np.random.seed(42)

# Generate random departure and arrival times (in hours)
num_records = 100
departure_times = np.random.randint(0, 24, num_records)
arrival_times = (departure_times + np.random.randint(1, 6, num_records)) % 24

# Generate random energy consumed (in kWh)
energy_consumed = np.random.uniform(10, 100, num_records)
distance_travelled = np.random.uniform(5, 500, num_records)

# Create a data frame using the generated values
data = {
    'Departure_Time': departure_times,
    'Arrival_Time': arrival_times,
    'Energy_Consumed': energy_consumed,
    'Distance_Travelled': distance_travelled
}
df_dp = pd.DataFrame(data)

# Display the data frame
print(df_dp.head())


   Departure_Time  Arrival_Time  Energy_Consumed  Distance_Travelled
0               6             9        31.560570          195.623376
1              19            22        23.040538          426.312652
2              14            15        54.050748          161.876393
3              10            13        98.708541           88.898910
4               7            10        31.784974          280.616625


In [25]:
# Merge the data frame with the time-of-use prices data frame
df_merged = df.merge(df_tou_prices, left_on='Departure_Time', right_on='Hour', how='left')

# Calculate the revenue for each record
base_fee = 0.99
df_merged['Revenue'] = base_fee + (df_merged['Energy_Consumed'] * df_merged['Price'])

# Display the merged data frame with the revenue column
print(df_merged.head())


   Departure_Time  Arrival_Time  Energy_Consumed  Hour  Price    Revenue
0               6             9        31.560570     6   0.39  13.298622
1              19            22        23.040538    19   0.59  14.583918
2              14            15        54.050748    14   0.51  28.555882
3              10            13        98.708541    10   0.51  51.331356
4               7            10        31.784974     7   0.39  13.386140


In [26]:
import pandas as pd

# Create an empty list to store the prices and base fees
prices_and_base_fees = []

# Loop through all the hours in a day (0-23)
for hour in range(24):
    price = 0.5
    base_fee = 0.99
    prices_and_base_fees.append([hour, price, base_fee])

# Create a data frame using the prices and base fees list
df_sched = pd.DataFrame(prices_and_base_fees, columns=['Hour', 'Price', 'Base_Fee'])

# Display the data frame
print(df_sched)


    Hour  Price  Base_Fee
0      0    0.5      0.99
1      1    0.5      0.99
2      2    0.5      0.99
3      3    0.5      0.99
4      4    0.5      0.99
5      5    0.5      0.99
6      6    0.5      0.99
7      7    0.5      0.99
8      8    0.5      0.99
9      9    0.5      0.99
10    10    0.5      0.99
11    11    0.5      0.99
12    12    0.5      0.99
13    13    0.5      0.99
14    14    0.5      0.99
15    15    0.5      0.99
16    16    0.5      0.99
17    17    0.5      0.99
18    18    0.5      0.99
19    19    0.5      0.99
20    20    0.5      0.99
21    21    0.5      0.99
22    22    0.5      0.99
23    23    0.5      0.99


In [39]:
dt=1

SOCmin= 0.2
SOCmax=0.8
Pmax = 11*10**3

## Optimizer Function

This function should ideally take in the chargemode preference and the driver profile, which contains information about the car efficiency etc, and output ideal time intervals for charging. This could be a good way for people to better understand how to create saving behaviours.

In [40]:
def optimizer(SOC0, E, c, N, chargemode, Qcap, efficiency):
    # SOC0 is initial state of charge, 
    # E is energy needed in the charge,
    # c is the cost of charging in one day ie from hours 0-24
    # N is the number of data points
    # chargemode is whatever is specified ie naive vs scheduled
    # Qcap is battery capacity
    # efficiency is specified efficiency
    
    P = Variable(N)
    SOC= Variable(N+1)
    
    #defining obj function
    if chargemode=="sched":
        objective= Minimize(dt* c.T @P)
    elif chargemode=="naive":
        objective= Maximize(dt*np.ones([1, N+1]) @ SOC*1000)
    
    constraints = [SOC[0]==SOC0]
    
    # SOC should be equal to the required energy but if this is more than 
    # SOCMax then just set goal to SOCMax
    if not E/(Qcap) + SOCmin > SOCmax:
        constraints += [SOC[N] >= E/ Qcap + SOCmin]
    else: 
        constraints += [SOC[N] >= SOCmax]
    
    # Now we need to figure out the state of charge at each time point
    for k in range(N):
        constraints += [SOC[k+1] == SOC[k]+(dt/Qcap)*P[k]*efficiency]
        
        constraints += [0 <= P[k], P[k]<=Pmax]
        
    for k in range(N):
        constraints += [SOCmin <= SOC[k], SOC[k]<= SOCmax]
    
    
    prob= Problem(objective, constraints)
    prob.solve(verbose=True, max_iters=100, abstol=1e-9)
    if prob.status not in ["unbounded", "infeasible"]:
        cost= np.sum(np.multiply(c, P.value/1000))
    else:
        cost=float("inf")
    
    return(cost, P.value, SOC.value)
        
        
    

In [41]:
def case(chargemode, elecprices, car, verbose=True):
    sim_df= pd.DataFrame(0, index=np.arange(np.shape(df_sched)[0]), columns=['costhr', 'P', 'SOC'])
    # df_DE is a df of electricity prices
    sim_df.index = df_sched.index
    
    
    ## Car dynamics 
    if car=='ModelS':
        avg_cons = 152 # Avg consumption from EV database
        efficiency= 152/190 #190 is rated consumption
        Qcap= 95
        energy_cons = df_dp['Distance_Travelled']*avg_cons
        df_dp['Energy Consumption']= energy_cons
    if car=='Mustang':
        avg_cons = 152 # Avg consumption from EV database
        efficiency= 152/173 #190 is rated consumption
        Qcap= 91
        energy_cons = df_dp['Distance_Travelled']*avg_cons
        df_dp['Energy Consumption']= energy_cons
    if car=='Ioniq':
        avg_cons = 127 # Avg consumption from EV database
        efficiency= 127/151 #190 is rated consumption
        Qcap= 74
        energy_cons = df_dp['Distance_Travelled']*avg_cons
        df_dp['Energy Consumption']= energy_cons
    if car=='Kia':
        avg_cons = 140 # Avg consumption from EV database
        efficiency= 140/164 #190 is rated consumption
        Qcap= 64.8
        energy_cons = df_dp['Distance_Travelled']*avg_cons
        df_dp['Energy Consumption']= energy_cons
        
    if chargemode=="naive":
        df_prices = df_tou_prices
    elif chargemode== "sched":
        df_prices=  df_sched
    
    SOC0 = SOCmin
    for i in range(1, np.shape(df_dp)[0]):
        start = df_dp['Arrival_Time'][i-1]
        end = df_dp["Departure_Time"][i]
        E= df_dp["Energy Consumption"][i]
        c= np.array(df_prices[start:end]['Price'])
        N=len(c)
        if chargemode=="naive":
            (cost_i,P_i,SOC_i)=optimizer(SOC0, E, c, N, "naive", Qcap, efficiency)
        elif chargemode=="sched":
            (cost_i, P_i, SOC_i)=optimizer(SOC0, E, c, N, "sched", Qcap, efficiency)
        
        sim_df.loc[start:end, 'costhr']= np.multiply(c, P_i/1000)
        sim_df.loc[start:end,'P']=P_i
        sim_df.loc[start:end, 'SOC']=SOC_i
        SOCN = SOC_i[N]
        if not SOC_i[N]- E/Qcap <SOCmin:
            SOC0= SOC_i[N]- E/(Qcap)
        else:
            SOC0= SOCmin
        chargedelec= np.sum(P_i)
    
    return sim_df
    
    

In [42]:
a = case('sched', df_sched, 'Ioniq')

                                     CVXPY                                     
                                     v1.2.2                                    
(CVXPY) Apr 17 11:37:13 AM: Your problem has 21 variables, 52 constraints, and 0 parameters.
(CVXPY) Apr 17 11:37:13 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 17 11:37:13 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 17 11:37:13 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 17 11:37:13 AM: Compiling problem (target solver=ECOS).
(CVXPY) Apr 17 11:37:13 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

ValueError: Must have equal len keys and value when setting with an iterable